In [124]:
import requests

def cos(vect1, vect2):
    mag_vect1 = np.linalg.norm(vect1)
    mag_vect2 = np.linalg.norm(vect2)
    
    return np.dot(vect1,vect2) / (mag_vect1 * mag_vect2)

def list_most_similar_docs(model, vect):
    for doc_id, cos_sim in model.docvecs.most_similar(vect, topn=30):
        try:
            resp = requests.get('http://localhost:9200/tmdb/movie/{}?_source=overview,title'.format(doc_id)).json()
            print(resp['_source']['title'], cos_sim)
        except e:
            print(e)
            pass

In [143]:
from word2vec import prepare_model
from elasticsearch import Elasticsearch
es = Elasticsearch()
# window=10,negative=10,min_count=3
modelep20 = prepare_model(es=es, window=15, negative=10, 
                          vector_size=300, min_count=5,
                          epochs=20)

#modelep10 = prepare_model(es=es, window=15, negative=10, 
#                          vector_size=300, min_count=5,
#                          epochs=10)

#print("Training epochs=100")
#modelep100 = prepare_model(es=es, window=5, negative=10, 
#                           vector_size=300, min_count=5,
#                           epochs=100)

model = modelep20

Attempting to Load Preanalyzed Data
Training
training on 22055 docs
Saving


In [144]:
rambo = model.docvecs['7555']
star_wars = model.docvecs['11']
clone_wars = model.docvecs['12180']
empire_strikes_back = model.docvecs['1891']
return_of_the_jedi = model.docvecs['1892']
conan_the_destroyer = model.docvecs['9610']

In [146]:
model.wv.most_similar(["jedi"], topn=20)

[('darth', 0.7670223712921143),
 ('skywalk', 0.7347890138626099),
 ('vader', 0.6643474102020264),
 ('galact', 0.6498984098434448),
 ('anakin', 0.6397547721862793),
 ('emperor', 0.626189649105072),
 ('galaxi', 0.6145325303077698),
 ('knight', 0.5965883731842041),
 ('obi', 0.5741980671882629),
 ('ruler', 0.5601556301116943),
 ('legion', 0.5379891395568848),
 ('sultan', 0.5364062786102295),
 ('brainiac', 0.5347437262535095),
 ('defeat', 0.5179133415222168),
 ('warrior', 0.5130146145820618),
 ('pharaoh', 0.5097701549530029),
 ('richelieu', 0.50380539894104),
 ('merlin', 0.5035779476165771),
 ('protector', 0.500783383846283),
 ('despot', 0.50030517578125)]

In [147]:
model.wv.most_similar(["jedi"], topn=20)

[('darth', 0.7670223712921143),
 ('skywalk', 0.7347890138626099),
 ('vader', 0.6643474102020264),
 ('galact', 0.6498984098434448),
 ('anakin', 0.6397547721862793),
 ('emperor', 0.626189649105072),
 ('galaxi', 0.6145325303077698),
 ('knight', 0.5965883731842041),
 ('obi', 0.5741980671882629),
 ('ruler', 0.5601556301116943),
 ('legion', 0.5379891395568848),
 ('sultan', 0.5364062786102295),
 ('brainiac', 0.5347437262535095),
 ('defeat', 0.5179133415222168),
 ('warrior', 0.5130146145820618),
 ('pharaoh', 0.5097701549530029),
 ('richelieu', 0.50380539894104),
 ('merlin', 0.5035779476165771),
 ('protector', 0.500783383846283),
 ('despot', 0.50030517578125)]

In [148]:
model.wv.most_similar(["han_solo"], topn=40)

KeyError: "word 'han_solo' not in vocabulary"

In [130]:
import numpy as np

model.wv.most_similar([star_wars], topn=20)

[('han_solo', 0.46316754817962646),
 ('ottoman', 0.3639509677886963),
 ('skywalk', 0.34860068559646606),
 ('leaguer', 0.33654940128326416),
 ('droid', 0.3167843818664551),
 ('shackleton', 0.30565544962882996),
 ('oppressor', 0.3048733174800873),
 ('armageddon', 0.30078086256980896),
 ('ninja_turtl', 0.2923094630241394),
 ('entebb', 0.28573381900787354),
 ('concord', 0.2833603322505951),
 ('turk', 0.2757364511489868),
 ('manchu', 0.27319446206092834),
 ('hubbl', 0.2685926556587219),
 ('dirig', 0.26790666580200195),
 ('nemo', 0.26689231395721436),
 ('ruechang', 0.2664097547531128),
 ('everest', 0.26431891322135925),
 ('thrush', 0.2629425823688507),
 ('darth', 0.2584596276283264)]

In [111]:
from rp import random_projection

rp_vects = []
for i in range(0,50):
    vect, lhs, rhs = random_projection(model.docvecs.vectors_docs)
    rp_vects.append(vect)
    
def rp_sim(rp_vects, vect1, vect2):
    same_sign = 0
    for rp_vect in rp_vects:
        dp1 = np.dot(rp_vect, vect1)
        dp2 = np.dot(rp_vect, vect2)
        if dp1 > 0 and dp2 > 0:
            same_sign += 1
        elif dp1 < 0 and dp2 < 0:
            same_sign += 1
    return same_sign / len(rp_vects)

    

In [155]:
cos(star_wars, return_of_the_jedi)

0.69055754

In [225]:
from analyzed_docs import overview_text_no_stop
from elasticsearch import Elasticsearch

def get_movie_text(movie_id):
    resp = requests.get("http://localhost:9200/tmdb/movie/%s?_source=overview" % movie_id).json()
    es = Elasticsearch()
    return overview_text_no_stop(es, resp['_source']['overview'])


def get_text_vectors(terms):
    vects = []
    for term in terms:
        stopwords = ["a", "an", "and", "are", "as", "at", "be", "but", "by",
            "for", "if", "in", "into", "is", "it",
            "no", "not", "of", "on", "or", "such",
            "that", "the", "their", "then", "there", "these",
            "they", "this", "to", "was", "will", "with"]
        if term in model.wv:
            if term not in stopwords:
                vects.append(model.wv[term])
    return np.array(vects)
            

    
sw_avg = np.average(get_text_vectors(get_movie_text(11)), axis=0)
esb_avg = np.average(get_text_vectors(get_movie_text(1891)), axis=0)
rj_avg = np.average(get_text_vectors(get_movie_text(1892)), axis=0)
rb_avg = np.average(get_text_vectors(get_movie_text(7555)), axis=0)
co_avg = np.average(get_text_vectors(get_movie_text(9610)), axis=0)
cd_avg = np.average(get_text_vectors(get_movie_text(11977)), axis=0)

cos(rb_avg, rj_avg)

0.61420316

In [229]:
model.wv.most_similar([cd_avg], topn=20)

[('tournament', 0.5479854345321655),
 ('competit', 0.5235376358032227),
 ('prestigi', 0.5121024250984192),
 ('firm', 0.502556324005127),
 ('caddi', 0.5013620853424072),
 ('scholarship', 0.4951714277267456),
 ('contest', 0.49273645877838135),
 ('compet', 0.48250657320022583),
 ('colleg', 0.4787086844444275),
 ('coach', 0.4691489040851593),
 ('succe', 0.451945424079895),
 ('club', 0.44660183787345886),
 ('promot', 0.44326022267341614),
 ('competitor', 0.44222819805145264),
 ('he', 0.44177770614624023),
 ('posit', 0.44119346141815186),
 ('exam', 0.44059622287750244),
 ('job', 0.43727046251296997),
 ('instructor', 0.42816585302352905),
 ('school', 0.4090803265571594)]

In [210]:
list_most_similar_docs(model, [sw_avg])

Ten Tall Men 0.44316595792770386
Deathstalker 0.4228644371032715
Family Guy Presents: It's a Trap! 0.40969473123550415
Threads of Destiny 0.4064246714115143
Star Wars: Episode III - Revenge of the Sith 0.4062676429748535
Hercules Against the Mongols 0.38845205307006836
Conan the Destroyer 0.3862627446651459
The Three Musketeers 0.3849222660064697
Yellowstone Kelly 0.3694799840450287
Star Wars: Episode II - Attack of the Clones 0.36654961109161377
Revolt of the Praetorians 0.3592129349708557
The Swan Princess: Escape from Castle Mountain 0.35879823565483093
Lady Bloodfight 0.35753387212753296
Captain America: The Winter Soldier 0.356174111366272
Seven Slaves Against the World 0.35602104663848877
Jarhead 2: Field of Fire 0.3552103042602539
The Divine Weapon 0.3544091582298279
Dr. No 0.353972464799881
Starship Apocalypse 0.3537600040435791
Aquaman 0.3509889245033264
Ninja 4: Siege 0.35021495819091797
The Lives of a Bengal Lancer 0.34963685274124146
The Saint 0.349540650844574
The Three St

In [18]:
#  dissimilar
list_most_similar_docs(model, vect=np.negative(star_wars))

In [16]:
np.dot(the_dinner_game, star_wars)

-0.07022353

In [17]:
np.dot(vect, castle_in_the_sky)

0.06100053463596857

In [31]:
np.dot(rambo, star_wars)

0.13079807

In [18]:
rp_sim(rp_vects, star_wars, conan_the_destroyer)

0.84

In [19]:
rp_sim(rp_vects, star_wars, empire_strikes_back)

0.76

In [20]:
rp_sim(rp_vects, star_wars, castle_in_the_sky)

0.76

In [21]:
rp_sim(rp_vects, star_wars, rambo)

0.78

In [22]:
rp_sim(rp_vects, star_wars, the_dinner_game)

0.38

In [18]:
dir(model)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__gt__',
 '__hash__',
 '__ignoreds',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__numpys',
 '__recursive_saveloads',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__scipys',
 '__setattr__',
 '__sizeof__',
 '__slotnames__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_adapt_by_suffix',
 '_check_input_data_sanity',
 '_check_training_sanity',
 '_clear_post_train',
 '_do_train_epoch',
 '_do_train_job',
 '_get_job_params',
 '_get_offsets_and_start_doctags_for_corpusfile',
 '_get_thread_working_mem',
 '_job_producer',
 '_load_specials',
 '_log_epoch_end',
 '_log_epoch_progress',
 '_log_progress',
 '_log_train_end',
 '_raw_word_count',
 '_save_specials',
 '_set_train_params',
 '_smart_save',
 '_train_epoch',
 '_train_epoch_corpusfile',
 '_update_job_params',
 '_worker_loop',
 '_worker_loop_